In [ ]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
# -*- coding: utf-8 -*-
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
warnings.filterwarnings("ignore")  # 选择过滤警告
import scipy.io as scio
import scipy
import statsmodels.tsa.api as smt
import statsmodels.tsa.api as smt
from statsmodels.tsa.stattools import coint
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import warnings


import datetime
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels import regression
from six import StringIO
#导入pca
from sklearn.decomposition import PCA
from sklearn.svm import SVC,SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

import matplotlib.dates as mdates
import matplotlib.pyplot as plt

import seaborn as sns
import os

pd.set_option('display.max_rows',500)
pd.set_option('display.min_rows',500)

In [ ]:
# 读取数据
data=pd.read_excel('data.xlsx',index_col='dates')
data['自由流通市值']=data['自由流通股本']*data['收盘价']
price=data['收盘价']
op=data['开盘价']
dates=data.index
dates=dates[(dates>'20100101') & (dates<'20210101')] #回测2010-2020数据
factors=data[['cpi_month', 'pmi', 'ind_growth_yoy', 'fixed_assets_investment',
       'consistency_idx','early_warning_idx',
       'foreign','m2', 'm1', 'retail_sin',
        'gdp_sin','commodity_house_idx',
       '十年国债', '成交额', '换手率', '自由流通市值', 'PE(TTM)', '股息率', 'PB(LR)',
       '一致预测净利润同比', '一致预测每股收益(FY1)', 'macd', 'kdj',
       'rsi', 'atr', 'ROE(平均)', '归母净利润同比增长率', 'EPS', '营业收入同比增长率']]
# 预测周期
m=19
# 训练周期
k=235
#tag值为一个月后的涨幅
price_ch=price.shift(-m)/price-1

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(price)
plt.legend(['万得全A指数收盘价'])

In [ ]:
len(['cpi_month', 'pmi', 'ind_growth_yoy', 'fixed_assets_investment',
       'consistency_idx','early_warning_idx',
       'foreign','m2', 'm1', 'retail_sin',
        'gdp_sin','commodity_house_idx',
       '十年国债', '成交额', '换手率', '自由流通市值', 'PE(TTM)', '股息率', 'PB(LR)',
       '一致预测净利润同比', '一致预测每股收益(FY1)', 'macd', 'kdj',
       'rsi', 'atr', 'ROE(平均)', '归母净利润同比增长率', 'EPS', '营业收入同比增长率'])

In [ ]:
# 小波变换处理收盘价
import pywt # 小波分析
import scipy.fftpack as fftpack # 希尔伯特变换

In [ ]:
#对指数时间序列进行小波分解，小波函数选择为db8，分解层个数为4个，即一个主要趋势，和四个细节成分
#四个参数分别 对应 输入的信号 ，小波名称， 信号扩展模式， 分解阶次  
res_4 = pywt.wavedec(price_ch.dropna(),wavelet='db8',mode='smooth',level=4) 
col_name = ['主趋势'] + [f'细节{i}' for i in range(1,5)]
res4_df = pd.DataFrame(res_4,index=col_name).T
res4_df.head()

In [ ]:
res4_df.iloc[:, 1:].plot(subplots=True, layout=(2, 2), figsize=(18, 8), sharex=False)
plt.show()
#从上图可以看到，相对而言，细节3和4更像是高频的噪声信号，故对这两个细节进行阈值降噪处理。
#这里涉及到阈值选择的问题。 通常来说，阈值选择有：1、无偏风险估计阈值(rigrsure) 2、固定阈值 
#3、启发式阈值 4、极大极小阈值 这些方法，在这里选择rigsure方法来确定阈值

In [ ]:
# 信号去噪
class DenoisingThreshold(object):
    '''
    获取小波去噪的阈值
    1. CalSqtwolog 固定阈值准则(sqtwolog)
    2. CalRigrsure 无偏风险估计准则（rigrsure）
    3. CalMinmaxi 极小极大准则（ minimaxi）
    4. CalHeursure
    参考：https://wenku.baidu.com/view/63d62a818762caaedd33d463.html
    对股票价格等数据而言，其信号频率较少地与噪声重叠因此可以选用sqtwolog和heursure准则，使去噪效果更明显。 
    但对收益率这样的高频数据，尽量采用保守的 rigrsure 或 minimaxi 准则来确定阈值，以保留较多的信号。
    '''
    def __init__(self,signal: np.array):
        self.signal = signal
        self.N = len(signal)
    # 固定阈值准则(sqtwolog)
    @property
    def CalSqtwolog(self) -> float:
        return np.sqrt(2 * np.log(self.N))
   
    # 无偏风险估计准则（rigrsure）
    @property
    def CalRigrsure(self)->float:
        N = self.N
        signal = np.abs(self.signal)
        signal = np.sort(signal)
        signal = np.power(signal, 2)
        risk_j = np.zeros(N)
        for j in range(N):
            if j == 0:
                risk_j[j] = 1 + signal[N - 1]
            else:
                risk_j[j] = (N - 2 * j + (N - j) *
                             (signal[N - j]) + np.sum(signal[:j])) / N
        k = risk_j.argmin()
        return np.sqrt(signal[k])
    # 极小极大准则（ minimaxi）
    @property
    def CalMinmaxi(self)->float:
        if self.N > 32:
            # N>32 可以使用minmaxi阈值 反之则为0
            return 0.3936 + 0.1829 * (np.log(self.N) / np.log(2))
        else:
            return 0
    @property
    def GetCrit(self)->float:
        return np.sqrt(np.power(np.log(self.N) / np.log(2), 3) * 1 / self.N)
    @property
    def GetEta(self)->float:
        return (np.sum(np.abs(self.signal)**2) - self.N) / self.N
    #混合准则（heursure）
    @property
    def CalHeursure(self):
        if self.GetCrit > self.GetEta:
            #print('推荐使用sqtwolog阈值')
            return self.CalSqtwolog
        else:
            #print('推荐使用 Min(sqtwolog阈值,rigrsure阈值)')
            return min(self.CalRigrsure,self.CalSqtwolog)

In [ ]:
#对于高频的信号进行阈值处理
for j in [3,4]:     
    signal = res_4[j]
    threshold = DenoisingThreshold(signal).CalRigrsure 
    res_4[j] = pywt.threshold(signal, threshold, 'soft')
rec_hs = pywt.waverec(res_4, 'db8')

In [ ]:
#得到小波处理后的股价
try:
    price_ch_new=pd.Series(rec_hs,index=price_ch.dropna().index)
except:
    price_ch_new=pd.Series(rec_hs[1:],index=price_ch.dropna().index)
price_ch_new.name='收盘价'
plt.figure(figsize=(15,4))
plt.plot(price_ch)
plt.plot(price_ch_new)
plt.legend(['原收益率曲线','小波降噪后曲线'])

In [ ]:
#是否使用小波降噪
price_ch=price_ch_new

In [ ]:
# 因子MA处理
factors['cpi_month']=factors['cpi_month'].rolling(120).mean()
factors['consistency_idx']=factors['consistency_idx'].rolling(60).mean()
factors['early_warning_idx']=factors['early_warning_idx'].rolling(60).mean()
factors['retail_sin']=factors['retail_sin'].rolling(60).mean()
factors['m1']=factors['m1'].rolling(60).mean()
factors['m2']=factors['m1'].rolling(60).mean()
factors['gdp_sin']=factors['gdp_sin'].rolling(120).mean()
factors['十年国债']=factors['十年国债'].rolling(60).mean()
factors['自由流通市值']=factors['自由流通市值']
factors['PE(TTM)']=factors['PE(TTM)'].rolling(20).mean()
factors['股息率']=factors['股息率'].rolling(60).mean()
factors['PB(LR)']=factors['PB(LR)'].rolling(120).mean()
factors['一致预测净利润同比']=factors['一致预测净利润同比'].rolling(60).mean()
factors['一致预测每股收益(FY1)']=factors['一致预测每股收益(FY1)'].rolling(60).mean()
factors['营业收入同比增长率']=factors['营业收入同比增长率'].rolling(60).mean()

In [ ]:
#去极值标准化
def mad(factor):
    me=np.median(factor)
    mad=np.median(abs(factor-me))
    up=me+1.4826*mad*3
    down=me-1.4826*mad*3
    factor=np.where(factor>up,up,factor)
    factor=np.where(factor<down,down,factor)
    return factor
def stand(factor):
    mean=factor.mean()
    std=factor.std()
    return (factor-mean)/std
factors=factors.apply(lambda x: mad(x),axis=0)
factors=factors.apply(lambda x: stand(x),axis=0)

In [ ]:
# 股价涨跌幅平稳性检验
plt.figure(figsize=(15,4))
plt.title('平稳性检验')
plt.plot(price_ch)
plt.show()
smt.adfuller(price_ch.dropna())

In [ ]:
#根据corr结果选择每个因子的滞后期
# factors['cpi_month']=factors['cpi_month'].shift(40)
# factors['fixed_assets_investment']=factors['fixed_assets_investment'].shift(120)
# factors['early_warning_idx']=factors['early_warning_idx'].shift(60)
factors['retail_sin']=factors['retail_sin'].shift(40)
factors['十年国债']=factors['十年国债'].shift(20)
factors['成交额']=factors['成交额'].shift(20)
factors['换手率']=factors['换手率'].shift(20)
factors['ind_growth_yoy']=factors['ind_growth_yoy'].shift(20)
factors['自由流通市值']=factors['自由流通市值'].shift(20)
factors['一致预测每股收益(FY1)']=factors['一致预测每股收益(FY1)'].shift(60)
factors['macd']=factors['macd'].shift(20)
factors['kdj']=factors['kdj'].shift(20)
factors['rsi']=factors['rsi'].shift(20)
factors['ROE(平均)']=factors['ROE(平均)'].shift(40)
factors['gdp_sin']=factors['gdp_sin'].shift(60)
factors['EPS']=factors['EPS'].shift(20)
factors['归母净利润同比增长率']=factors['归母净利润同比增长率'].shift(60)

In [ ]:
factors=factors[['cpi_month','ind_growth_yoy', 'fixed_assets_investment','retail_sin','consistency_idx','early_warning_idx','commodity_house_idx',
       'foreign','成交额', '换手率', '自由流通市值','十年国债', 'gdp_sin','PE(TTM)', '股息率',
                 '一致预测净利润同比','一致预测每股收益(FY1)','ROE(平均)', '归母净利润同比增长率','EPS',
                 '营业收入同比增长率','PB(LR)','macd', 'kdj','rsi']]

In [ ]:
# SVM
from sklearn.decomposition import PCA
from sklearn.svm import SVC,SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [ ]:
#基于因子值与一个月后收益率关系，生成tag值
# tag=price_ch.dropna().apply(lambda x: np.where(x>=0.01,1,np.where(x<-0.01,-1,0)))
tag=price_ch.dropna().apply(lambda x: np.where(x>=0,1,-1))
#统一时间维度
factors=factors[factors.index>='2010-01-01'] #选取回测起使时期
dates=tag.index & factors.index
tag=tag.loc[dates]
factors=factors.loc[dates]

In [ ]:
def pca_deal(data,value):
    '''
    data: 二维表
    value: 方差解释超过哪个阈值后剩下的不要'''
    pca = PCA()
    pca.fit(data)
    ctb=pca.explained_variance_ratio_ #查看主成分贡献度
    n=(ctb.cumsum()>value).tolist().index(True) #获取累计贡献度到一定程度的索引值
    pca = PCA(n_components=n)
    pca.fit(data)
    return pca.fit_transform(data)

In [ ]:
def svm_model(x,y,rolling_len=k,rolling_period=m):
    '''
    x: 因子数据
    y: tag数据，与x的长度必须一致，不允许有空
    rolling_len: 每次rolling训练集加上测试集的数据长度
    rolling_period: 每次rolling向前滚动的长度，也是测试集的长度
    '''
    regressor=SVC(decision_function_shape='ovo')
    parameters = {'kernel':['rbf','linear'],
                  'C':[0.1,1,3],
                 'gamma':[0.0001,0.001,0.01,1]}
    dtL=[] #记录每个预测数据的日期，每一次rolling加入一个y_test的日期list
    predL=[] #记录每个预测值，每次加入一个预测值的list
    paraL=[] #记录每次rolling的最佳参数值，每次加入一个字典信息
    train_accL=[]
    test_accL=[]
    train_aucL=[]
    test_aucL=[]
    cv_accL=[]
    train_f1L=[]
    test_f1L=[]
    #独热编码
    enc = OneHotEncoder()
    enc.fit([[-1],[0],[1]])
    #设置rolling模型
    for i in range(0,len(y)-rolling_len,rolling_period): #i为每次rolling的起使日期
        # 每次rolling划分训练集和测试集
        temp_x=x.iloc[i:(i+rolling_len)].dropna(axis=1)
        temp_y=y.iloc[i:(i+rolling_len)]
        # PCA 降维处理
        temp_x=pca_deal(temp_x,0.99)
        #划分训练集测试集
        X_train, X_test, y_train, y_test = train_test_split(temp_x, temp_y, test_size=rolling_period/rolling_len,shuffle=False)
        #跑模型并记录结果
        clf = GridSearchCV(regressor,parameters,scoring=['accuracy'],refit='accuracy',cv=5)
        clf.fit(X_train,y_train)
        X_test_pred=clf.predict(X_test)
        X_train_pred=clf.predict(X_train)
        dtL.append(y_test.index)
        predL.append(X_test_pred)
        paraL.append(clf.best_params_)
        #记录模型效果，分别为模型内，模型外的accuracy
        train_accL.append(metrics.accuracy_score(y_train,X_train_pred))
        test_accL.append(metrics.accuracy_score(y_test,X_test_pred))
        cv_accL.append(clf.cv_results_['mean_test_accuracy'].mean())
        #记录f1 score
        train_f1L.append(metrics.f1_score(y_train,X_train_pred))
        test_f1L.append(metrics.f1_score(y_test,X_test_pred))
        #auc 如果是多维的要先转成独热编码格式
#         a=enc.transform(y_train.values.reshape(-1,1)).toarray()
#         b=enc.transform(X_train_pred.reshape(-1,1)).toarray()
#         c=enc.transform(y_test.values.reshape(-1,1)).toarray()
#         d=enc.transform(X_test_pred.reshape(-1,1)).toarray()
        # 如果标签全部一样，会报错，这时返回none
        
        try:
            train_aucL.append(metrics.roc_auc_score(y_train,X_train_pred))
        except ValueError:
            train_aucL.append(None)
        try:
            test_aucL.append(metrics.roc_auc_score(y_test,X_test_pred))
        except ValueError:
            test_aucL.append(None)
    return dtL,predL,paraL,train_accL,test_accL,cv_accL,train_aucL,test_aucL,train_f1L,test_f1L

import time
s=time.clock()
dtL,predL,paraL,train_accL,test_accL,cv_accL,train_aucL,test_aucL,train_f1L,test_f1L=svm_model(factors,tag)
e=time.clock()
print((e-s)/60)

In [ ]:
#结果成表
summary_list=pd.DataFrame(index=[x[0] for x in dtL])
summary_list['C']=[x['C'] for x in paraL]
summary_list['gamma']=[x['gamma'] for x in paraL]
summary_list['kernel']=[x['kernel'] for x in paraL]
summary_list['train_acc']=train_accL
summary_list['test_acc']=test_accL
summary_list['train_auc']=train_aucL
summary_list['test_auc']=test_aucL
summary_list['cv_acc']=cv_accL
summary_list['train_f1']=train_f1L
summary_list['test_f1']=test_f1L

In [ ]:
summary_list

In [ ]:
# 参数使用频率统计
summary_list['kernel'].value_counts(1)

In [ ]:
#画图
plt.figure(figsize=(15,4))
plt.plot(summary_list[['train_acc','test_acc','cv_acc']])
plt.plot([summary_list.index[0],summary_list.index[-1]],[0.5,0.5],linestyle='--')
plt.legend(['训练集准确率','测试集准确率','交叉验证集准确率'])

In [ ]:
plt.figure(figsize=(15,4))
summary_list['test_auc']=summary_list['test_auc'].bfill()
plt.plot(summary_list[['train_auc','test_auc']])
plt.legend(['训练集AUC','测试集AUC'])

In [ ]:
# f1 score
plt.figure(figsize=(15,4))
plt.plot(summary_list[['train_f1','test_f1']])
plt.plot([summary_list.index[0],summary_list.index[-1]],[0.5,0.5],linestyle='--')
plt.legend(['训练集f1','测试集f1'])

In [ ]:
# 二值分类的tag与因子值的相关性表
#数据准备以及对齐
dates=[]
for i in dtL:
    for j in i:
        dates.append(j)
tag_pred=[]
for i in predL:
    for j in i:
        tag_pred.append(j)
tag_pred=pd.Series(tag_pred,index=dates)
tag_merge=tag_pred.to_frame().join(tag.to_frame(),how='inner')
tag_merge.columns=['预测值','真实值'] #这个是日频tag预测值与真实值
factors2=factors.loc[tag_merge.index]
factors_name=factors2.columns
data=factors2.join(tag_merge) #merge factor后
dates=data.index
#每年回测一次
n=500
dates2=[] #记录每次测试的开始时间
result_pred=pd.DataFrame()
result_true=pd.DataFrame()

for i in range(30,len(factors2.index)-int(n/2),int(n/2)):
    dates2.append(dates[i])
    temp_data=data.iloc[i:i+n].dropna(axis=1)
    temp_names=temp_data.columns[:-2]
    temp_true=pd.Series(None,index=data.columns[:-2])
    temp_pred=pd.Series(None,index=data.columns[:-2])
    for j in temp_names:
        temp_pred[j]=abs((temp_data[j].to_frame().join(pd.get_dummies(temp_data.iloc[:,-2]))).corr().iloc[0,1])
        temp_true[j]=abs((temp_data[j].to_frame().join(pd.get_dummies(temp_data.iloc[:,-1]))).corr().iloc[0,1])
#         print(dates[i])
#         display(corr_table)
#         display(temp_data[j].to_frame().join(pd.get_dummies(temp_data.iloc[:,-2:])).head())
    result_pred=pd.concat([result_pred,temp_pred.T],axis=1)
    result_true=pd.concat([result_true,temp_true.T],axis=1)
result_pred.columns=dates2
result_true.columns=dates2  
result_pred=result_pred.T
result_true=result_true.T

In [ ]:
from seaborn import heatmap
def drawHM(data,vmax):
    '''
    输入index为因子值，columns为时间的二维表
    vmax: 最大刻度
    '''
    plt.figure(figsize=(15,8))
    data.columns=[str(x)[:4] for x in data.columns]
    cmap = sns.cubehelix_palette(start = 1.5, rot = 3, gamma=0.8, as_cmap = True) #调颜色
    heatmap(data,cmap=cmap,linewidths = 0.05,annot=True,vmax=vmax)
drawHM(result_pred.T,1)
drawHM(result_true.T,1)

In [ ]:
# 每个因子的解释程data度大小，用tag=1的因子值除以去tag=-1的
n=500
factors_explain_pred=pd.DataFrame()
factors_explain_true=pd.DataFrame()
dates3=[]
for i in range(30,len(factors2.index)-int(n/2),int(n/2)):
    dates3.append(dates[i])
    temp_data=data.iloc[i:i+n].dropna(axis=1)
    temp_names=temp_data.columns[:-2]
    temp_true=pd.Series(None,index=data.columns[:-2])
    temp_pred=pd.Series(None,index=data.columns[:-2])
    for j in temp_names:
        pos_pred=temp_data[temp_data['预测值']==1][j].mean()
        neg_pred=temp_data[temp_data['预测值']==-1][j].mean()
        pos_true=temp_data[temp_data['真实值']==1][j].mean()
        neg_true=temp_data[temp_data['真实值']==-1][j].mean()
#         maxP=np.max([pos_pred,neg_pred])
#         minP=np.min([pos_pred,neg_pred])
#         maxT=np.max([pos_true,neg_true])
#         minT=np.min([pos_true,neg_true])
        temp_pred[j]=abs(pos_pred-neg_pred)
        temp_true[j]=abs(pos_true-neg_true)
    factors_explain_pred=pd.concat([factors_explain_pred,temp_pred],axis=1)
    factors_explain_true=pd.concat([factors_explain_true,temp_true],axis=1)
factors_explain_pred.columns=dates3
factors_explain_true.columns=dates3  
factors_explain_pred=factors_explain_pred.T
factors_explain_true=factors_explain_true.T

In [ ]:
#tag=1组/tag=0组-1然后取绝对值，反映因子方向的强弱
drawHM(factors_explain_pred.T,2)
drawHM(factors_explain_true.T,2)

In [ ]:
# 策略统计指标
dt=[x[0] for x in dtL]
dv=tag_merge.loc[dt]
metrics.accuracy_score(dv['真实值'],dv['预测值'])
# metrics.f1_score(dv['真实值'],dv['预测值'])
print('样本内准确率    ：',summary_list['train_acc'].mean())
print('样本外准确率    ：',summary_list['test_acc'].mean())
print('交叉验证集准确率：',summary_list['cv_acc'].mean())
print('样本内AUC       ：',summary_list['train_auc'].mean())
print('样本外AUC       ：',summary_list['test_auc'].mean())
print('样本内f1 score  ：',summary_list['train_f1'].mean())
print('样本外f1 score  ：',summary_list['test_f1'].mean())
print('月频策略预测成功概率 :',metrics.accuracy_score(dv['真实值'],dv['预测值']))
print('月频策略预测上涨成功概率 :',metrics.accuracy_score(dv[dv['预测值']==1]['真实值'],dv[dv['预测值']==1]['预测值']))
print('月频策略预测下跌成功概率 :',metrics.accuracy_score(dv[dv['预测值']==-1]['真实值'],dv[dv['预测值']==-1]['预测值']))

In [ ]:
data=pd.read_excel('data.xlsx',index_col='dates')

In [ ]:
# 择时模型
#所需要的数据以及统一格式
ret=price/price.shift(1)-1
dates=ret.index & op.index & tag_pred.index
ret=ret[dates] # 日收益率
op=op[dates] # 开盘价
cl=price[dates] #收盘价
data=pd.read_excel('data.xlsx',index_col='dates').loc[dates]
macd=data['macd']
atr=data['atr']
tag_pred=tag_pred[dates]
tag_true=tag[dates]

In [ ]:
#最简单的，tag=1下个月做多，tag=-1的退仓或做空
def basic_stg(tag_pred,ret,short,period=m):
    # period 调仓周期
    # short bool是否允许做空
    ls_stg=[1] #初始资金
    count=0
    signal=0
    dates=tag_pred.index
    for i in range(0,len(dates)-1): #第i日开始做预测，然后第i+1日开始调仓
        if count%period==0: # 每个周期获取一次预测信号
            signal=tag_pred[i]
        if signal==1: #做多
            temp_ls_stg=ls_stg[i]*(1+ret[i+1])
            ls_stg.append(temp_ls_stg)
        if signal==-1:
            if short: #做空
                temp_ls_stg=ls_stg[i]*(1-ret[i+1])
                ls_stg.append(temp_ls_stg)
            else: #什么也不做
                temp_ls_stg=ls_stg[i]
                ls_stg.append(temp_ls_stg)
        count+=1
    return pd.Series(ls_stg,index=tag_pred.index)
l_stg=basic_stg(tag_pred,ret,False)
ls_stg=basic_stg(tag_pred,ret,True)

In [ ]:
def calIndex(data,capital):
    init=capital
    record=[]
    for ret in data.values:
        init*=(1+ret)
        record.append(init)
    record=pd.DataFrame(record,index=data.index)
    return record[0]
bench=calIndex(ret,1)

In [ ]:
# 网格交易策略+预测
# 之前最简单的策略加上：1 周期第一天设置+-0.5，+-1ATR作为四根网格线，每碰到一根线卖半仓，每根线效果不重复触发。周期末清仓
# 2 macd 死叉清仓 3 乌云压顶清仓
grid_stg=pd.Series(None,index=dates) #资金持有量
grid_stg_num=pd.Series(None,index=dates) #持仓量
atr_high=pd.Series(None,index=dates)
atr_low=pd.Series(None,index=dates)
count=0
money=1 #设置初始资金
num=0 #设置初始仓位
signal=0 #涨跌预测信号
sell1,sell2,sell3,sell4=0,0,0,0 #初始化四个网格线，从小到大
w1,w2,w3,w4,w5,w6=-2.5,-1.5,-1,2,3.5,5 # 网格线宽度参数
for date in dates:
    cdt=0
    temp_cl=cl[date] #今日收盘价
    if count%19==0:  #月初调仓
        count+=1
        if tag_pred[date]==1: #如果下个月看涨
            signal=1
            num+=money/temp_cl #调仓日全仓买入指数
            money=0
            num_record=num #记录期初的持股数量
            grid_stg[date],grid_stg_num[date]=money,num #记录
            temp_atr=atr[date] #调仓日atr
            sell1,sell2,sell3,sell4,sell5,sell6=temp_cl+w1*temp_atr,temp_cl+w2*temp_atr,temp_cl+w3*temp_atr,temp_cl+w4*temp_atr,temp_cl+w5*temp_atr,temp_cl+w6*temp_atr
            sell2_sign,sell3_sign,sell4_sign,sell5_sign=0,0,0,0 #设置中间网格线的触发记录 0未触发 1触发
            atr_high.iloc[count-1:count+19],atr_low.iloc[count-1:count+19]=sell6,sell1
            continue
        else:
            signal=-1
            #清仓
            atr_high.iloc[count-1:count+19],atr_low.iloc[count-1:count+19]=None,None
            money+=temp_cl*num
            num=0
            grid_stg[date],grid_stg_num[date]=money,num #记录
            continue
    count+=1
    if signal==1: #看涨信号，开始执行策略
        temp_cl=cl[date] #当日收盘价
        if temp_cl>sell4 and sell4_sign==0 and num>=num_record*0.25:
            sell4_sign=1
            #卖出1/4
            money+=num_record*0.25*sell4
            num-=num_record*0.25
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif temp_cl>sell5 and sell5_sign==0 and num>=num_record*0.5:
            sell5_sign=1
            #卖出1/4
            money+=num_record*0.5*sell5
            num-=num_record*0.5
            grid_stg[date],grid_stg_num[date]=money,num #记录
            
        elif temp_cl>sell6 and round(num,6)!=0:
            #清仓
            money+=num*sell6
            num=0
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif temp_cl<sell3 and sell3_sign==0 and num>=num_record*0.25:
            sell3_sign=1
            money+=num_record*0.25*sell3
            num-=num_record*0.25
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif temp_cl<sell2 and sell2_sign==0 and num>=num_record*0.5:
            sell2_sign=1
            money+=num_record*0.5*sell2
            num-=num_record*0.5
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif temp_cl<sell1 and round(num,6)!=0:
            #清仓
            money+=num*sell1
            num=0
            grid_stg[date],grid_stg_num[date]=money,num #记录
#         elif macd[count-3]<0 and macd[count-4]>0:
#             money+=num*temp_cl
#             num=0
#             grid_stg[date],grid_stg_num[date]=money,num #记录
        else:
            grid_stg[date],grid_stg_num[date]=money,num #记录
    else: #看跌信号，观望
        grid_stg[date],grid_stg_num[date]=money,num #记录
grid_stg_sum1=grid_stg+grid_stg_num*cl

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(grid_stg_sum1)
plt.figure(figsize=(15,5))
plt.plot(cl)
plt.plot(atr_high)
plt.plot(atr_low)
plt.legend(['收盘价','上网格线','下网格线'])

In [ ]:
# 网格交易策略
grid_stg=pd.Series(None,index=dates) #资金持有量
grid_stg_num=pd.Series(None,index=dates) #持仓量
atr_high=pd.Series(None,index=dates)
atr_low=pd.Series(None,index=dates)
count=0
money=1 #设置初始资金
num=0 #设置初始仓位
signal=0 #涨跌预测信号
sell1,sell2,sell3,sell4=0,0,0,0 #初始化四个网格线，从小到大
w1,w2,w3,w4,w5,w6=-2.5,-1.5,-1,2,3.5,5 # 网格线宽度参数
for date in dates:
    cdt=0
    temp_cl=cl[date] #今日收盘价
    if count%19==0:  #月初调仓
        count+=1
        if True: #如果下个月看涨
            num+=money/temp_cl #调仓日全仓买入指数
            money=0
            num_record=num #记录期初的持股数量
            grid_stg[date],grid_stg_num[date]=money,num #记录
            temp_atr=atr[date] #调仓日atr
            sell1,sell2,sell3,sell4,sell5,sell6=temp_cl+w1*temp_atr,temp_cl+w2*temp_atr,temp_cl+w3*temp_atr,temp_cl+w4*temp_atr,temp_cl+w5*temp_atr,temp_cl+w6*temp_atr
            sell2_sign,sell3_sign,sell4_sign,sell5_sign=0,0,0,0 #设置中间网格线的触发记录 0未触发 1触发
            atr_high[date],atr_low[date]=sell6,sell1
            continue
    count+=1
    atr_high[date],atr_low[date]=sell3,sell2
    if True:
        temp_cl=cl[date] #当日收盘价
        if temp_cl>sell4 and sell4_sign==0 and num>=num_record*0.25:
            sell4_sign=1
            #卖出1/4
            money+=num_record*0.25*sell4
            num-=num_record*0.25
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif temp_cl>sell5 and sell5_sign==0 and num>=num_record*0.5:
            sell5_sign=1
            #卖出1/4
            money+=num_record*0.5*sell5
            num-=num_record*0.5
            grid_stg[date],grid_stg_num[date]=money,num #记录
            
        elif temp_cl>sell6 and round(num,6)!=0:
            #清仓
            money+=num*sell6
            num=0
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif temp_cl<sell3 and sell3_sign==0 and num>=num_record*0.25:
            sell3_sign=1
            money+=num_record*0.25*sell3
            num-=num_record*0.25
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif temp_cl<sell2 and sell2_sign==0 and num>=num_record*0.5:
            sell2_sign=1
            money+=num_record*0.5*sell2
            num-=num_record*0.5
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif temp_cl<sell1 and round(num,6)!=0:
            #清仓
            money+=num*sell1
            num=0
            grid_stg[date],grid_stg_num[date]=money,num #记录
        elif macd[count-3]<0 and macd[count-4]>0:
            money+=num*temp_cl
            num=0
            grid_stg[date],grid_stg_num[date]=money,num #记录
        else:
            grid_stg[date],grid_stg_num[date]=money,num #记录
grid_stg_sum2=grid_stg+grid_stg_num*cl

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(l_stg)
plt.plot(bench)
plt.plot(grid_stg_sum2)
plt.plot(grid_stg_sum1)
plt.legend(['多头策略','指数净值','网格交易策略','网格交易策略+模型预测'])

plt.figure(figsize=(15,4))
plt.plot(bench)
plt.plot(l_stg)
plt.plot(ls_stg)
plt.legend(['指数基准','多头策略','多空策略'])

In [ ]:
def MaxDrawdown(return_list):
    '''最大回撤率'''
    i = np.argmax((np.maximum.accumulate(return_list) - return_list) / np.maximum.accumulate(return_list))  # 结束位置
    if i == 0:
        return 0
    j = np.argmax(return_list[:i])  # 开始位置
    return (return_list[j] - return_list[i]) / (return_list[j])
def calRet(data:pd.Series()):
    # 输入净值曲线
    ret=(data/data.shift(1)-1).dropna()
    df=pd.DataFrame()
    df["年化收益率"]=[np.power(data[-1]/data[0],k/len(data))-1]
    df["累计收益率"]=[data[-1]/data[0]]
    df['年化波动率']=[np.sqrt(k)*ret.values.std()]
    df["夏普比率"]=[df["年化收益率"][0]/df["年化波动率"][0]]
    df['最大回撤率']=[MaxDrawdown(data)]
    return df.T
result=pd.DataFrame()
for line in [bench,l_stg,ls_stg,grid_stg_sum2,grid_stg_sum1]:
    result=pd.concat([result,calRet(line)[0]],axis=1)
result.columns=['万得全A基准','多头组合','多空组合','网格交易策略','网格交易策略+模型预测']
result

In [ ]:
# 画图
plt.figure(figsize=(15,5))
x=ls_stg.index
plt.plot(price[x])
for i in range(0,len(x)-m,m):
    if tag_pred[i]==1:
        plt.axvspan(x[i+1], x[i+m],alpha=0.3, color='g')
    if tag_pred[i]==-1:
        plt.axvspan(x[i+1], x[i+m],alpha=0.3, color='r')

